In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available


# Hyperparameter search space
search_space = {
    'learning_rate': [0.00005],
    'window_size': [ 5],
    'hidden_dim': [256],
    'n_layers': [11],
    'batch_evaluation_frequency': [4]
}

#Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256, 'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 0.41702309250831604}
#              'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

C:\Users\pitr7\AppData\Local\Temp\ipykernel_9356\3789981942.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_9356\3789981942.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_window = torch.tensor(x_val[

Iteration 1000, Train Loss: 1.0508795976638794, Validation Loss: 2.2516603469848633
Iteration 2000, Train Loss: 2.080430507659912, Validation Loss: 2.2376272678375244
Iteration 3000, Train Loss: 0.23619112372398376, Validation Loss: 2.238978147506714
Iteration 4000, Train Loss: 0.3774866461753845, Validation Loss: 2.219364881515503
Iteration 5000, Train Loss: 0.3024561405181885, Validation Loss: 2.222336769104004
Iteration 6000, Train Loss: 0.16863176226615906, Validation Loss: 2.225689172744751
Iteration 7000, Train Loss: 0.228369802236557, Validation Loss: 2.2307190895080566
Iteration 8000, Train Loss: 0.3323546051979065, Validation Loss: 2.2233834266662598


C:\Users\pitr7\AppData\Local\Temp\ipykernel_9356\3789981942.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)


Test Loss: 0.41863125562667847
Iteration 1000, Train Loss: 1.049631953239441, Validation Loss: 2.2504501342773438
Iteration 2000, Train Loss: 2.0760912895202637, Validation Loss: 2.2360541820526123
Iteration 3000, Train Loss: 0.2388816773891449, Validation Loss: 2.237497329711914
Iteration 4000, Train Loss: 0.3783760666847229, Validation Loss: 2.2188658714294434
Iteration 5000, Train Loss: 0.3043081760406494, Validation Loss: 2.221346616744995
Iteration 6000, Train Loss: 0.16978013515472412, Validation Loss: 2.224794864654541
Iteration 7000, Train Loss: 0.22999408841133118, Validation Loss: 2.2298755645751953
Iteration 8000, Train Loss: 0.3340439796447754, Validation Loss: 2.2227566242218018
Test Loss: 0.4180432856082916
Iteration 1000, Train Loss: 1.0496667623519897, Validation Loss: 2.250458240509033
Iteration 2000, Train Loss: 2.0759952068328857, Validation Loss: 2.236020088195801
Iteration 3000, Train Loss: 0.2388318032026291, Validation Loss: 2.237531900405884
Iteration 4000, Trai

In [14]:
print("Dimensions of last_window:", last_window.size())


Dimensions of last_window: torch.Size([1, 5, 7])


Batch size: 1
Time steps (window size): 5
Features (input size): 7

In [43]:
# Print the last window
print("Last Training Window:")
print("Input Features:")
print(x_window)
print("True Output:")
print(y_window)

Last Training Window:
Input Features:
tensor([[[-0.4585, -0.4632, -0.4611,  0.2122, -0.4199, -0.3131,  0.9702],
         [-0.6824, -0.6819, -0.6794, -0.5100, -0.4514,  0.6253, -0.4263],
         [-0.6759, -0.6739, -0.6726, -0.4536, -0.4510,  0.2627,  0.1182],
         [-0.6574, -0.6620, -0.6573, -0.4358, -0.4505, -0.6401, -0.1456],
         [-0.6137, -0.6046, -0.6089, -0.6418, -0.4574,  1.9724, -0.3608]]])
True Output:
tensor([[-0.4617],
        [-0.6794],
        [-0.6724],
        [-0.6614],
        [-0.6015]])


In [46]:
# Before the loop where x_window and y_window are defined

# Create inverse scalers
inverse_scaler_x = StandardScaler()
inverse_scaler_y = StandardScaler()

# Fit inverse scalers to the original data
inverse_scaler_x.fit(x_data)
inverse_scaler_y.fit(y_data)

# Inside the loop after defining x_window and y_window

# Convert x_window and y_window to numpy arrays
x_window_np = x_window.view(-1, input_size).numpy()
y_window_np = y_window.numpy()

# Inverse transform to original scale
x_window_original_scale = inverse_scaler_x.inverse_transform(x_window_np)
y_window_original_scale = inverse_scaler_y.inverse_transform(y_window_np)

# Print the last training window in the original scale
print("Last Training Window in Original Scale:")
print("Input Features:")
print(x_window_original_scale)
print("True Output:")
print(y_window_original_scale)


Last Training Window in Original Scale:
Input Features:
[[ 5.1718800e+01  5.1928802e+01  5.0551201e+01  2.4210100e+07
   1.5550994e+00 -7.6999998e-01  5.8811258e+08]
 [ 2.2198395e+01  2.2847994e+01  2.2052801e+01  4.6991990e+06
   3.4270084e-01  1.8700000e+00  2.0789803e+08]
 [ 2.3049606e+01  2.3911997e+01  2.2937605e+01  6.2216000e+06
   3.5679996e-01  8.5000002e-01  3.5614374e+08]
 [ 2.5491196e+01  2.5491205e+01  2.4931196e+01  6.7037000e+06
   3.7870014e-01 -1.6900002e+00  2.8430387e+08]
 [ 3.1248001e+01  3.3129597e+01  3.1247999e+01  1.1384001e+06
   1.1379994e-01  5.6599998e+00  2.2571920e+08]]
True Output:
[[51.2988  ]
 [22.6128  ]
 [23.542404]
 [24.981598]
 [32.883194]]


In [45]:

# Inverse transform to get original scales
x_window_original_scale = scaler_x.inverse_transform(x_window.view(params['window_size'], -1).numpy())
y_window_original_scale = scaler_y.inverse_transform(y_window.numpy())
outputs_original_scale = scaler_y.inverse_transform(outputs.numpy())

# Print the last window in original scale
print("Last Training Window (Original Scale):")
print("Input Features:")
print(x_window_original_scale)
print("True Output:")
print(y_window_original_scale)
print("Predicted Output:")
print(outputs_original_scale)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [41]:
# Print the original values of the last window in the test set with the last 5 entries and 7 features
last_window_features = data.iloc[-params['window_size']:][['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
original_last_window_values = scaler_y.inverse_transform(y_test[-params['window_size']:].reshape(-1, 1)).flatten()

print("\nLast Window in the Test Set - Original Values:")
print("Last 5 Entries - Features:")
print(last_window_features)
print("\nOriginal Values:")
print(original_last_window_values)



Last Window in the Test Set - Original Values:
Last 5 Entries - Features:
[[ 1.9333000e+02  1.9444000e+02  1.9292000e+02  3.7283200e+07
   1.9335890e+02  4.5000000e-01  5.2369165e+07]
 [ 1.9367000e+02  1.9564000e+02  1.9332000e+02  4.7471900e+07
   1.9423770e+02  4.5000000e-01  5.2206220e+07]
 [ 1.9602000e+02  1.9720000e+02  1.9255000e+02  4.7460200e+07
   1.9295940e+02 -6.6000000e-01  5.2018390e+07]
 [ 1.9467000e+02  1.9663000e+02  1.9414000e+02  4.8291400e+07
   1.9556590e+02  1.3500000e+00  5.2115595e+07]
 [ 1.9606000e+02  1.9649000e+02  1.9526000e+02  3.8824100e+07
   1.9618510e+02  3.2000000e-01  4.9803320e+07]]

Original Values:
[614.4796    31.628807  39.748802  35.369606 563.9004  ]


In [ ]:
# Invert the scaling on the initial last window values
last_window_original_scale = torch.tensor(scaler_x.inverse_transform(last_window.squeeze().numpy()), dtype=torch.float32)

# Print the initial last window values in the original scale
print("Initial Last Window from the Test Set (Original Scale):")
print(last_window_original_scale.numpy())
print()


In [42]:
# Print the original values of the last window in the test set with the last 5 entries and 7 features
last_window_features_scaled = x_test[-params['window_size']:]
last_window_target_scaled = y_test[-params['window_size']:]

# Reverse the scaled features
original_last_window_features = scaler_x.inverse_transform(last_window_features_scaled.numpy())

# Reverse the scaled target variable
original_last_window_target = scaler_y.inverse_transform(last_window_target_scaled.numpy().reshape(-1, 1)).flatten()

print("\nLast Window in the Test Set - Original Values:")
print("Last 5 Entries - Scaled Features:")
print(last_window_features_scaled.numpy())
print("\nOriginal Values - Features:")
print(original_last_window_features)
print("\nOriginal Values - Target Variable:")
print(original_last_window_target)


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [32]:
print("Initial Last Window from the Test Set:")
print(last_window.squeeze().numpy())
print()

Initial Last Window from the Test Set:
[[ 7.09588468e-01  6.94068849e-01  7.08556354e-01 -1.27636999e-01
  -2.99862981e-01 -2.56235391e-01  7.99318731e-01]
 [-6.33621991e-01 -6.37571454e-01 -6.29014373e-01 -6.79282546e-01
  -4.57734585e-01  1.22595763e+00 -1.09407175e+00]
 [-5.18926620e-01 -5.19055605e-01 -5.15072644e-01 -6.30042851e-01
  -4.52214748e-01  2.69818723e-01 -6.40534580e-01]
 [ 1.76104784e+00  1.73794806e+00  1.74404860e+00 -7.56878555e-02
  -1.91887781e-01 -3.87748927e-01  5.77984333e-01]
 [-6.61148906e-01 -6.64778769e-01 -6.60588562e-01 -6.33377790e-01
  -4.55435306e-01 -3.94857764e-01 -6.09052598e-01]
 [-7.27204919e-01 -7.26226747e-01 -7.29571342e-01 -4.12115514e-01
  -4.54135150e-01 -2.83674407e+00 -1.59285605e-01]
 [-5.54949462e-01 -5.56623518e-01 -5.55312634e-01 -6.36760890e-01
  -4.53156799e-01  7.67437458e-01 -6.42346621e-01]
 [-6.97171748e-01 -6.87311113e-01 -6.95122838e-01  1.40832931e-01
  -4.51625675e-01  3.63229990e+00  3.22337365e+00]
 [-3.63365680e-01 -3.7046

In [34]:
# Convert predicted values back to the original scale using inverse transformation
predicted_values = np.array(predicted_values).reshape(-1, 1)

# Invert the scaling on predicted values
predicted_values_original_scale = scaler_y.inverse_transform(predicted_values)

# Print or use the predicted values as needed
print("Predicted Values in Original Scale:")
print(predicted_values_original_scale)


Predicted Values in Original Scale:
[[116.5111242]]


In [31]:

# Choose the number of steps to predict
n_steps = 1  # You can adjust this based on your needs

# Extract the last window from the test set
last_window = torch.tensor(x_test[-window_size:], dtype=torch.float32).view(1, window_size, input_size)

# Initialize the hidden state
hidden = model.init_hidden(1)

# Generate predictions for the next 'n_steps' in a loop
predicted_values = []

for _ in range(n_steps):
    with torch.no_grad():
        # Forward pass to get the next prediction
        prediction, hidden = model(last_window, hidden)

        # Append the prediction to the list of predicted values
        predicted_values.append(prediction.item())

        # Update the last window for the next iteration
        last_window = torch.cat([last_window[:, 1:, :], prediction.view(1, 1, -1).expand(-1, -1, last_window.size(2))], dim=1)

# Convert predicted values back to the original scale using inverse transformation
predicted_values = np.array(predicted_values).reshape(-1, 1)
predicted_values_original_scale = scaler_y.inverse_transform(predicted_values)

# Print or use the predicted values as needed
print("Predicted Values in Original Scale:")
print(predicted_values_original_scale)


Predicted Values in Original Scale:
[[116.5111242]]


C:\Users\pitr7\AppData\Local\Temp\ipykernel_9356\2961749471.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_window = torch.tensor(x_test[-window_size:], dtype=torch.float32).view(1, window_size, input_size)
